In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# MLflow Classification Recipe Notebook

This notebook runs the MLflow Classification Recipe on Databricks and inspects its results. For more information about the MLflow Classification Recipe, including usage examples, see the [Classification Recipe overview documentation](https://mlflow.org/docs/latest/recipes.html#classification-recipe) the [Classification Recipe API documentation](https://mlflow.org/docs/latest/python_api/mlflow.recipes.html#module-mlflow.recipes.classification.v1.recipe).

In [23]:
from mlflow.recipes import Recipe

r = Recipe(profile="local")
r.clean()

2023/04/13 15:13:04 INFO mlflow.recipes.recipe: Creating MLflow Recipe 'mlflow-recipes-titanic' with profile: 'local'


In [24]:
# Display the DAGs (Directed Acyclic Graph)
# r.inspect()

In [25]:
r.run("ingest")

2023/04/13 15:13:05 INFO mlflow.recipes.step: Running step ingest...


In [26]:
r.run("split")

2023/04/13 15:13:06 INFO mlflow.recipes.utils.execution: ingest: No changes. Skipping.


Run MLFlow Recipe step: split
2023/04/13 15:13:07 INFO mlflow.recipes.step: Running step split...


In [27]:
training_data = r.get_artifact("training_data")
training_data.sample(5)

,Survived,Pclass,Sex,SibSp,Parch,Fare,Age,Embarked
PassengerId,,,,,,,,
31,0,1,male,0,0,27.7208,40.0,C
783,0,1,male,0,0,30.0000,29.0,S
263,0,1,male,1,1,79.6500,52.0,S
776,0,3,male,0,0,7.7500,18.0,S
60,0,3,male,5,2,46.9000,11.0,S


In [28]:
r.run("transform")

2023/04/13 15:13:08 INFO mlflow.recipes.utils.execution: ingest, split: No changes. Skipping.


Run MLFlow Recipe step: transform
2023/04/13 15:13:09 INFO mlflow.recipes.step: Running step transform...


Name,Type
Survived,int64
Pclass,int64
Sex,object
SibSp,int64
Parch,int64
Fare,float64
Age,float64
Embarked,object
Name,Type
onehot__Pclass_1,float64


In [29]:
r.run("train")

2023/04/13 15:13:11 INFO mlflow.recipes.utils.execution: ingest, split, transform: No changes. Skipping.


Run MLFlow Recipe step: train
2023/04/13 15:13:12 INFO mlflow.recipes.step: Running step train...
2023/04/13 15:13:12 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/04/13 15:13:12 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2023/04/13 15:13:12 INFO mlflow.recipes.steps.train: Training data has less than 5000 rows, skipping rebalancing.
INFO: GPU available: False, used: False
INFO  [lightning.pytorch.utilities.rank_zero] GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO  [lightning.pytorch.utilities.rank_zero] TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO  [lightning.pytorch.utilities.rank_zero] IPU available: False, usin

Metric,training,validation
accuracy_score,0.832865,0.746835
example_count,712,79
f1_score,0.774194,0.615385
false_negatives,71,10
false_positives,48,10
precision_score,0.809524,0.615385
recall_score,0.741818,0.615385
score,0.832865,0.746835
true_negatives,389,43
true_positives,204,16


In [30]:
trained_model = r.get_artifact("model")
print(trained_model)

mlflow.pyfunc.loaded_model:
  artifact_path: train/model
  flavor: mlflow.pyfunc.model
  run_id: 67808e89856a4545ae4eece1d1793afb



In [31]:
from sklearn.metrics import classification_report

test_data = r.get_artifact("training_data")
y_test = test_data["Survived"]
pred = trained_model.predict(test_data)

print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.85      0.89      0.87       437
           1       0.81      0.74      0.77       275

    accuracy                           0.83       712
   macro avg       0.83      0.82      0.82       712
weighted avg       0.83      0.83      0.83       712



In [32]:
r.run("evaluate")

2023/04/13 15:14:03 INFO mlflow.recipes.utils.execution: ingest, split, transform, train: No changes. Skipping.


Run MLFlow Recipe step: evaluate
2023/04/13 15:14:04 INFO mlflow.recipes.step: Running step evaluate...
2023/04/13 15:14:04 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/04/13 15:14:04 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2023/04/13 15:14:07 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2023/04/13 15:14:07 INFO mlflow.models.evaluation.default_evaluator: The evaluation dataset is inferred as binary dataset, positive label is 1, negative label is 0.
2023/04/13 15:14:07 INFO mlflow.models.evaluation.default_evaluator: Shap explainer _PatchedKernelExplainer is used.

100%|██████████| 10/10 [00:00<00:00, 36.93it/s]
2023/04/13 15:14:17 WARNING mlflow.utils

Metric,validation,test
accuracy_score,0.746835,0.800000
example_count,79,100.000000
f1_score,0.615385,0.756098
false_negatives,10,10.000000
false_positives,10,10.000000
precision_score,0.615385,0.756098
recall_score,0.615385,0.756098
score,0.746835,0.800000
true_negatives,43,49.000000
true_positives,16,31.000000


In [33]:
r.run("register")

2023/04/13 15:14:20 INFO mlflow.recipes.utils.execution: ingest, split, transform, train, evaluate: No changes. Skipping.


Run MLFlow Recipe step: register
2023/04/13 15:14:20 INFO mlflow.recipes.step: Running step register...
2023/04/13 15:14:21 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/04/13 15:14:21 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2023/04/13 15:14:21 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/04/13 15:14:21 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Registered model 'titanic-local' already exists. Creating a new version of this model...
2023/04/13 15:14:21 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for 

In [34]:
# Only works with Spark
# r.run("ingest_scoring")
# r.run("predict")